<h1 style="color: green;"> Applied Capstone Week3 Assignment </h1>

<h2 style="color: blue;"> Part I:  Web Scraping </h2>

## Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    
<font size=3>
    
1. <a href="#item1">Retrieving data from the webpage provided</a>

2. <a href="#item2">Creating a list of tuples from the webpage data</a>

3. <a href="#item3">Wrangling data</a>

4. <a href="#item4">Exporting data to CSV file for possible future use</a>

5. <a href="#item5">Shape of final dataframe</a>

</font>
</div>

### Retrieving data from the webpage provided

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh

In [2]:
# Scraping data from a wikipedia source page

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Create a handle, page, to handle the contents of the website
page = requests.get(url)

# Store the contents of the website under doc
doc = lh.fromstring(page.content)

# Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

### Creating a list of tuples from the webpage data 

In [3]:
# Create empty list
wikidata=[]
i=0
# For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    name = name[:-1]
    wikidata.append((name,[]))

wikidata

[('Postal code', []), ('Borough', []), ('Neighborhood', [])]

In [4]:
# Filling the empty list created earlier
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        # Remove the ending character '\n'
        data = data[:-1]

        #Append the data to the empty list of the i'th column
        wikidata[i][1].append(data)
        #Increment i for the next column
        i+=1


### Wrangling data 

- creating a dictionary from the list obtained 
- creating a dataframe from this dictionary
- getting rid of the last line that contain an irrelevant 'Borough' (i.e. 'Canadian postal codes')
- saving a version of the dataframe
- ignoring all _'Not assigned'_ boroughs

In [6]:
# Creating a dictionary from the list of tuples created above
candict = dict(wikidata)

# Creating the dataframe from dictionary 'tordict'
can_df = pd.DataFrame.from_dict(candict)

# There's an irrelevant extra 'Borough' containing 'Canadian postal codes'. We get rid of it by this :
can_df = can_df.iloc[:180,]

# Replacing the slash in 'Neighborhood' values
can_df = can_df.applymap(lambda x: x.replace(" / ",", "))

# Ignoring the 'Not assigned' boroughs
all_df = can_df
can_df = can_df[can_df['Borough'] != 'Not assigned']


In [7]:
can_df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Exporting data to CSV file for possible future use

In [ ]:
can_df.to_csv("Canadian_postal_codes.csv")

### Displaying shape of the final dataframe

In [8]:
can_df.shape

(103, 3)